In [ ]:
import json
from copy import deepcopy

from hscap.constants import *
from hscap.data import read_hcris_gj

# Note! This is a work in progress. Mainly I consider this mock data.

In [ ]:
geojson = read_hcris_gj()

In [ ]:
geojson['features'][0]

In [ ]:
new_gj = deepcopy(geojson)
for feature in new_gj['features']:
    new_props = {}
    
    licensed_icu_beds = feature['properties']['ICU Total Staffed Beds']
    for scenario in scenarios:
        beds_field = scenarios[scenario][BEDS]['field']
        beds_ratio = scenarios[scenario][BEDS]['ratio']
        beds_surge_percentages = scenarios[scenario][BEDS]['surge percentage']
        
        beds_raw = feature['properties'][beds_field]
        icu_beds = beds_raw * beds_ratio
        icu_beds_min = icu_beds * beds_surge_percentages[0]
        icu_beds_max = icu_beds * beds_surge_percentages[1]        
        
        new_props[construct_beds_field_name(scenario, BOUND_LOWER)] = round(icu_beds_min)
        new_props[construct_beds_field_name(scenario, BOUND_UPPER)] = round(icu_beds_max)
        
        # Ventilator need
        new_props[construct_vents_field_name(scenario, BOUND_LOWER)] = round(max(0, icu_beds_min - licensed_icu_beds))
        new_props[construct_vents_field_name(scenario, BOUND_UPPER)] = round(max(0, icu_beds_max - licensed_icu_beds))
        
        for staff_need in STAFF:
            available_count_ratio = scenarios[scenario][staff_need]['available count ratio']
            patients_per = scenarios[scenario][staff_need]['patients per']
            available_percentage = scenarios[scenario][staff_need]['available percentage']
            
            if icu_beds <= 0:
                min_need, max_need = 0, 0
            else:
                min_need, max_need = None, None
            
                for patient_need in [icu_beds_min, icu_beds_max]:
                    for patients in patients_per:
                        for surge_percentage in available_percentage:
                            need_count = (patient_need * surge_percentage) / patients
                            if min_need is None or need_count < min_need:
                                min_need = need_count
                            if max_need is None or max_need < need_count:
                                max_need = need_count
            new_props[construct_staff_field_name(
                scenario, staff_need, BOUND_LOWER)] = round(max(1, min_need))
            new_props[construct_staff_field_name(
                scenario, staff_need, BOUND_UPPER)] = round(max(1, max_need))
            
    new_props['Name'] = feature['properties']['HOSP10_Name']
    new_props['State'] = feature['properties']['State']
    new_props['County'] = feature['properties']['County']
    
    feature['properties'] = new_props
    
print(json.dumps(new_gj['features'][0], indent=4))

In [ ]:
open('../data/hscap_data_by_facility.geojson', 'w').write(json.dumps(new_gj, indent=2))